In [2]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
from __future__ import unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import cv2
import numpy as np

from keras import backend as K
import tensorflow as tf
from keras.models import *
from keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization,Activation, add, concatenate
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [3]:
def residual_block(X, filters, strides):

    X_shortcut = X

    X = Conv2D(filters = filters[0], kernel_size = (3,3), padding = 'same', strides = strides[0], kernel_initializer = glorot_uniform(seed = 0))(X)
    X = Activation(activation = 'relu')(X)
    X = BatchNormalization()(X)
    

    X = Conv2D(filters = filters[1], kernel_size = (3,3), padding = 'same', strides = strides[1], kernel_initializer = glorot_uniform(seed = 0))(X)
    X = Activation(activation = 'relu')(X)
    X = BatchNormalization()(X)
    
    
    X_shortcut = Conv2D(filters = filters[1], kernel_size = (1,1), strides = strides[0], kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    X = add([X, X_shortcut])
    X = Activation(activation = 'relu')(X)

    return X

In [4]:
def encoder(X):

    output = []

    X_shortcut = X

    X = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', strides = (1, 1), kernel_initializer = glorot_uniform(seed = 0))(X)
    X = Activation(activation = 'relu')(X)
    X = BatchNormalization()(X)

    X = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', strides = (1, 1), kernel_initializer = glorot_uniform(seed = 0))(X)


    X_shortcut = Conv2D(filters = 64, kernel_size = (1, 1), strides = (1, 1), kernel_initializer = glorot_uniform(seed = 0))(X)
    X_shortcut = BatchNormalization()(X_shortcut)

    X = add([X, X_shortcut])
    #first branch of the decoder
    output.append(X)

    X = residual_block(X, [128,128], [(2, 2), (1,1)])
    #second branch of the decoder
    output.append(X)

    X = residual_block(X, [256, 256], [(2, 2), (1, 1)])
    #third branch of the decoder
    output.append(X)

    return output


In [5]:
def decoder(X, encoder_output):

    X = UpSampling2D(size = (2, 2))(X)
    X = concatenate([X, encoder_output[2]], axis = 3)
    X = residual_block(X, [256, 256], [(1, 1), (1, 1)])

    X = UpSampling2D(size = (2, 2))(X)
    X = concatenate([X, encoder_output[1]], axis = 3)
    X = residual_block(X, [128, 128], [(1, 1), (1, 1)])

    X = UpSampling2D(size = (2, 2))(X)
    X = concatenate([X, encoder_output[0]], axis = 3)
    X = residual_block(X, [64, 64], [(1, 1), (1, 1)])

    return X

In [6]:
def build_model(input_shape):

    X = Input(shape = input_shape)

    encoder_output = encoder(X)

    X1 = residual_block(encoder_output[2], [512, 512], [(2, 2), (1,1)])

    X1 = decoder(X1, encoder_output)

    X1 = Conv2D(filters = 1, kernel_size = (1, 1), activation = 'sigmoid', kernel_initializer = glorot_uniform(seed = 0))(X1)

    return Model(input = X, output = X1)


In [11]:
model = build_model(input_shape = (512,512,4))

C:\Users\cisl\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  del sys.path[0]


In [ ]:

smooth = 1.

dropout_rate = 0.5

act = "relu"



def mean_iou(y_true, y_pred):

    prec = []

    for t in np.arange(0.5, 1.0, 0.05):

        y_pred_ = tf.to_int32(y_pred > t)

        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)

        K.get_session().run(tf.local_variables_initializer())

        with tf.control_dependencies([up_opt]):

            score = tf.identity(score)

        prec.append(score)

    return K.mean(K.stack(prec), axis=0)



# Custom loss function

def dice_coef(y_true, y_pred):

    smooth = 1.

    y_true_f = K.flatten(y_true)

    y_pred_f = K.flatten(y_pred)

    intersection = K.sum(y_true_f * y_pred_f)

    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)



def dice_coef_loss(y_true, y_pred):

    return 1. - dice_coef(y_true, y_pred)



def bce_dice_loss(y_true, y_pred):

    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)



# Evaluation metric: IoU

def compute_iou(img1, img2):



    img1 = np.array(img1)

    img2 = np.array(img2)



    if img1.shape[0] != img2.shape[0]:

        raise ValueError("Shape mismatch: the number of images mismatch.")

    IoU = np.zeros( (img1.shape[0],), dtype=np.float32)

    for i in range(img1.shape[0]):

        im1 = np.squeeze(img1[i]>0.5)

        im2 = np.squeeze(img2[i]>0.5)



        if im1.shape != im2.shape:

            raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")



        # Compute Dice coefficient

        intersection = np.logical_and(im1, im2)



        if im1.sum() + im2.sum() == 0:

            IoU[i] = 100

        else:

            IoU[i] = 2. * intersection.sum() * 100.0 / (im1.sum() + im2.sum())

        #database.display_image_mask_pairs(im1, im2)



    return IoU



# Evaluation metric: Dice

def compute_dice(im1, im2, empty_score=1.0):

    """

    Computes the Dice coefficient, a measure of set similarity.

    Parameters

    ----------

    im1 : array-like, bool

        Any array of arbitrary size. If not boolean, will be converted.

    im2 : array-like, bool

        Any other array of identical size. If not boolean, will be converted.

    Returns

    -------

    dice : float

        Dice coefficient as a float on range [0,1].

        Maximum similarity = 1

        No similarity = 0

        Both are empty (sum eq to zero) = empty_score



    Notes

    -----

    The order of inputs for `dice` is irrelevant. The result will be

    identical if `im1` and `im2` are switched.

    """

    im1 = np.asarray(im1).astype(np.bool)

    im2 = np.asarray(im2).astype(np.bool)



    if im1.shape != im2.shape:

        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")



    im_sum = im1.sum() + im2.sum()

    if im_sum == 0:

        return empty_score



    # Compute Dice coefficient

    intersection = np.logical_and(im1, im2)



    return 2. * intersection.sum() / im_sum

